In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Concatenate
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
import os
import re
from IPython.display import display

In [2]:
print("Krok 0: Konfiguracja środowiska")

# Tworzenie katalogów na artefakty i wyniki
ARTIFACTS_DIR = "Artifacts_Polska"
os.makedirs(ARTIFACTS_DIR, exist_ok=True)

# Nazwy plików
LOC_FILE = 'lokalizacja.csv'
DATA_FILE_1 = 'saleflats_2024_dateAdded_polska.csv'
DATA_FILE_2 = 'saleflats_2024_newestDate_polska.csv'
OUTPUT_FILE = 'Location_Polska.csv'

# Hiperparametry
MAX_TEXT_FEATURES = 20000

Krok 0: Konfiguracja środowiska


In [3]:
print("--- START: Budowa finalnego zbioru danych ---")

# --- Krok 1: Wczytanie i przygotowanie pliku lokalizacja.csv ---
print("Wczytywanie i przygotowywanie pliku lokalizacja.csv...")
try:
    df_lokalizacja_raw = pd.read_csv(
        'lokalizacja.csv', sep=',', header=None,
        names=['Id', 'ParentId', 'Name', 'AdditionalName', 'FullName'], dtype=str
    )
    df_lokalizacja = df_lokalizacja_raw.copy()
    df_lokalizacja['Id'] = pd.to_numeric(df_lokalizacja['Id'], errors='coerce')
    df_lokalizacja['ParentId'] = df_lokalizacja['ParentId'].replace('\\N', pd.NA)
    df_lokalizacja['ParentId'] = pd.to_numeric(df_lokalizacja['ParentId'], errors='coerce')
    df_lokalizacja.dropna(subset=['Id'], inplace=True)
    df_lokalizacja['Id'] = df_lokalizacja['Id'].astype(int)
    df_lokalizacja['ParentId'] = df_lokalizacja['ParentId'].astype('Int64')
    
    location_dict = df_lokalizacja.set_index('Id').to_dict('index')
    id_to_name = df_lokalizacja.set_index('Id')['Name'].to_dict()
    id_to_parent = df_lokalizacja.set_index('Id')['ParentId'].to_dict()
    
    # Tworzenie struktur do walidacji hierarchii
    miasta_ids = df_lokalizacja[df_lokalizacja['AdditionalName'].str.contains('Miasto', na=False)]['Id'].tolist()
    dzielnice_ids = df_lokalizacja[df_lokalizacja['AdditionalName'].str.contains('Dzielnica|Osiedle', na=False)]['Id'].tolist()
    city_to_districts = df_lokalizacja[df_lokalizacja['Id'].isin(dzielnice_ids)].groupby('ParentId')['Id'].apply(list).to_dict()
    district_to_streets = df_lokalizacja[df_lokalizacja['AdditionalName'] == 'Ulica'].groupby('ParentId')['Id'].apply(list).to_dict()
    
    print(f"Plik lokalizacja.csv wczytany. Mamy {len(location_dict)} unikalnych lokalizacji.")
except FileNotFoundError:
    print("BŁĄD KRYTYCZNY: Nie znaleziono pliku lokalizacja.csv")
    exit()

# --- Krok 2: Ekstrakcja danych z plików ofert za pomocą 'usecols' ---
# UWAGA: Indeksy kolumn mogą wymagać weryfikacji. Na podstawie przykładów i błędów,
# zakładamy, że plik ma więcej kolumn, a `locationPath` jest na końcu.
# Próbujemy znaleźć ostatnią kolumnę, ale jeśli to zawiedzie, użyjemy stałej.
try:
    # Szybki odczyt pierwszego wiersza, żeby znaleźć ostatnią kolumnę
    first_row = pd.read_csv('saleflats_2024_dateAdded_polska.csv', sep=',', header=None, nrows=1, on_bad_lines='skip')
    last_col_index = first_row.shape[1] - 1
    print(f"Automatycznie wykryto, że ostatnia kolumna ma indeks: {last_col_index}")
except:
    last_col_index = 64 # Wartość awaryjna
    print(f"Nie udało się wykryć ostatniej kolumny, używam wartości awaryjnej: {last_col_index}")

COLS_TO_EXTRACT = {
    3: 'Title', 4: 'Description', 5: 'Area', 6: 'Price',
    9: 'NumberOfRooms', 14: 'Floor', 15: 'Floors', 16: 'BuiltYear',
    last_col_index: 'locationPath' # Używamy wykrytego indeksu
}

all_offers_df = []
FILES = ['saleflats_2024_dateAdded_polska.csv', 'saleflats_2024_newestDate_polska.csv']
for filepath in FILES:
    print(f"\nEkstrakcja danych z pliku: {filepath}...")
    try:
        df_chunk = pd.read_csv(
            filepath, sep=',', header=None, on_bad_lines='skip',
            usecols=list(COLS_TO_EXTRACT.keys()), low_memory=False, dtype=str, encoding='utf-8'
        )
        df_chunk.rename(columns=COLS_TO_EXTRACT, inplace=True)
        all_offers_df.append(df_chunk)
        print(f"Udało się wyekstrahować {len(df_chunk)} wierszy.")
    except Exception as e:
        print(f"Błąd podczas ekstrakcji z {filepath}: {e}")

if not all_offers_df:
    print("BŁĄD KRYTYCZNY: Nie udało się wczytać żadnych danych z plików ofert.")
    exit()

df = pd.concat(all_offers_df, ignore_index=True)
df.dropna(subset=['locationPath'], inplace=True)
df.drop_duplicates(subset=['Title', 'Description', 'locationPath'], keep='first', inplace=True)
print(f"\nPołączono i uzyskano {len(df)} unikalnych ofert.")


# --- Krok 3: Parsowanie ścieżki i tworzenie kolumn docelowych ---
print("Parsowanie ścieżek i tworzenie kolumn docelowych...")
path_cols = ['Wojewodztwo_ID', 'Powiat_ID', 'Gmina_ID', 'Miasto_ID', 'Dzielnica_ID', 'PodDzielnica_ID', 'Ulica_ID']
split_paths = df['locationPath'].str.split(',')
path_df = pd.DataFrame(split_paths.tolist(), index=df.index)
if path_df.shape[1] > 7: path_df = path_df.iloc[:, :7]
while path_df.shape[1] < 7: path_df[path_df.shape[1]] = None
path_df.columns = path_cols
path_df = path_df.apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)
df = pd.concat([df.reset_index(drop=True), path_df.reset_index(drop=True)], axis=1)

df['target_district_id'] = np.where(df['PodDzielnica_ID'] != 0, df['PodDzielnica_ID'], df['Dzielnica_ID'])
df['target_city_id'] = df['Miasto_ID']
df['target_street_id'] = df['Ulica_ID']

# Filtrujemy, wymagając tylko, aby miasto było określone
df = df[df['target_city_id'] != 0].copy()
df.reset_index(drop=True, inplace=True)

if len(df) == 0:
    print("BŁĄD KRYTYCZNY: Po filtrowaniu nie pozostały żadne wiersze z określonym miastem.")
else:
    print(f"Liczba ogłoszeń z określonym miastem do treningu: {len(df)}")

--- START: Budowa finalnego zbioru danych ---
Wczytywanie i przygotowywanie pliku lokalizacja.csv...
Plik lokalizacja.csv wczytany. Mamy 398832 unikalnych lokalizacji.
Automatycznie wykryto, że ostatnia kolumna ma indeks: 52

Ekstrakcja danych z pliku: saleflats_2024_dateAdded_polska.csv...
Udało się wyekstrahować 793664 wierszy.

Ekstrakcja danych z pliku: saleflats_2024_newestDate_polska.csv...
Udało się wyekstrahować 755679 wierszy.

Połączono i uzyskano 773591 unikalnych ofert.
Parsowanie ścieżek i tworzenie kolumn docelowych...
Liczba ogłoszeń z określonym miastem do treningu: 709104


In [4]:
print("\nKrok 2: Inżynieria Cech (Uproszczona i Niezawodna)")
from sklearn.impute import SimpleImputer

# --- Przetwarzanie cech numerycznych (tylko pewne kolumny) ---
# ZMIANA: Skupiamy się tylko na 'Area' i 'Price', które na pewno są poprawne.
num_features = ['Area', 'Price']
for col in num_features:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Używamy SimpleImputer, aby uzupełnić ewentualne braki
imputer = SimpleImputer(strategy='median')
df[num_features] = imputer.fit_transform(df[num_features])

scaler = StandardScaler()
X_num = scaler.fit_transform(df[num_features])
joblib.dump(scaler, os.path.join(ARTIFACTS_DIR, 'scaler.joblib'))

# --- Przetwarzanie cech tekstowych (ze zredukowaną złożonością) ---
df['text_features'] = df['Title'].fillna('') + ' ' + df['Description'].fillna('')
df['text_features'] = df['text_features'].apply(lambda x: re.sub(r'\s+', ' ', x.lower()))

MAX_TEXT_FEATURES_REDUCED = 5000
vectorizer = TfidfVectorizer(max_features=MAX_TEXT_FEATURES_REDUCED, ngram_range=(1, 1))

print("Rozpoczynam wektoryzację tekstu ze zredukowanymi parametrami...")
X_text = vectorizer.fit_transform(df['text_features'])
joblib.dump(vectorizer, os.path.join(ARTIFACTS_DIR, 'vectorizer.joblib'))

# Aktualizujemy globalną zmienną
MAX_TEXT_FEATURES = MAX_TEXT_FEATURES_REDUCED

print(f"Przygotowano cechy: {X_text.shape[1]} tekstowych (TF-IDF) i {X_num.shape[1]} numerycznych.")
print(f"Kształt macierzy X_text: {X_text.shape}")
print(f"Kształt macierzy X_num: {X_num.shape}")


Krok 2: Inżynieria Cech (Uproszczona i Niezawodna)
Rozpoczynam wektoryzację tekstu ze zredukowanymi parametrami...
Przygotowano cechy: 5000 tekstowych (TF-IDF) i 2 numerycznych.
Kształt macierzy X_text: (709104, 5000)
Kształt macierzy X_num: (709104, 2)


In [5]:
print("\nKrok 3: Przygotowanie Celów (Target Variables) jako indeksy")

# Mapowanie ID na indeksy 0..N-1 (tak jak wcześniej)
city_ids = sorted(df['target_city_id'].unique())
district_ids = sorted(df['target_district_id'].unique())
street_ids = sorted(df['target_street_id'].unique())

if 0 not in district_ids: district_ids.insert(0,0)
if 0 not in street_ids: street_ids.insert(0,0)

city_id_map = {id: i for i, id in enumerate(city_ids)}
district_id_map = {id: i for i, id in enumerate(district_ids)}
street_id_map = {id: i for i, id in enumerate(street_ids)}

# Zapisanie mapowań
joblib.dump(city_id_map, os.path.join(ARTIFACTS_DIR, 'city_id_map.joblib'))
joblib.dump(district_id_map, os.path.join(ARTIFACTS_DIR, 'district_id_map.joblib'))
joblib.dump(street_id_map, os.path.join(ARTIFACTS_DIR, 'street_id_map.joblib'))

inv_city_id_map = {i: id for id, i in city_id_map.items()}
inv_district_id_map = {i: id for id, i in district_id_map.items()}
inv_street_id_map = {i: id for id, i in street_id_map.items()}
joblib.dump(inv_city_id_map, os.path.join(ARTIFACTS_DIR, 'inv_city_id_map.joblib'))
joblib.dump(inv_district_id_map, os.path.join(ARTIFACTS_DIR, 'inv_district_id_map.joblib'))
joblib.dump(inv_street_id_map, os.path.join(ARTIFACTS_DIR, 'inv_street_id_map.joblib'))

# KLUCZOWA ZMIANA: y to teraz wektory z indeksami, a nie macierze one-hot
y_city = df['target_city_id'].map(city_id_map).values
y_district = df['target_district_id'].map(district_id_map).values
y_street = df['target_street_id'].map(street_id_map).values

print(f"Przygotowano zmienne docelowe jako wektory indeksów. Liczba unikalnych klas:")
print(f"Miasta: {len(city_ids)}")
print(f"Dzielnice: {len(district_ids)}")
print(f"Ulice: {len(street_ids)}")


Krok 3: Przygotowanie Celów (Target Variables) jako indeksy
Przygotowano zmienne docelowe jako wektory indeksów. Liczba unikalnych klas:
Miasta: 6933
Dzielnice: 1940
Ulice: 28177


In [6]:
# Podział na zbiór treningowy i walidacyjny
# Ten kod teraz zadziała, bo nie operuje na gigantycznych macierzach y
X_text_train, X_text_val, X_num_train, X_num_val, y_city_train, y_city_val, y_district_train, y_district_val, y_street_train, y_street_val = train_test_split(
    X_text, X_num, y_city, y_district, y_street, test_size=0.2, random_state=42
)

print(f"Rozmiar zbioru treningowego: {X_text_train.shape[0]}")
print(f"Rozmiar zbioru walidacyjnego: {X_text_val.shape[0]}")

Rozmiar zbioru treningowego: 567283
Rozmiar zbioru walidacyjnego: 141821


In [7]:
print("\nKrok 4: Budowa i Kompilacja Modelu (Zoptymalizowana Architektura)")

# Wejścia
input_text = Input(shape=(MAX_TEXT_FEATURES,), name='text_input', sparse=True) 
input_num = Input(shape=(X_num_train.shape[1],), name='num_input')

# --- KLUCZOWA ZMIANA: Redukcja wymiarowości danych tekstowych ---
# Zamiast od razu przechodzić do dużej warstwy Dense(256),
# najpierw redukujemy 20 000 cech do mniejszej, gęstej reprezentacji (np. 128).
# To drastycznie zmniejsza zużycie pamięci w kolejnych warstwach.
x1 = Dense(128, activation='relu')(input_text)
x1 = Dropout(0.5)(x1) # Zwiększamy dropout, bo pierwsza warstwa jest teraz kluczowa

# Ścieżka numeryczna (bez zmian)
x2 = Dense(64, activation='relu')(input_num)
x2 = Dense(32, activation='relu')(x2)

# Połączenie
combined = Concatenate()([x1, x2])
z = Dense(256, activation='relu')(combined)
z = Dropout(0.5)(z)

# Głowy wyjściowe
output_city = Dense(len(city_ids), activation='softmax', name='city_output')(z)
output_district = Dense(len(district_ids), activation='softmax', name='district_output')(z)
output_street = Dense(len(street_ids), activation='softmax', name='street_output')(z)

# Finalny model
model = Model(inputs=[input_text, input_num], outputs=[output_city, output_district, output_street])

# Kompilacja (bez zmian, sparse_categorical_crossentropy jest poprawne)
model.compile(
    optimizer='adam',
    loss={
        'city_output': 'sparse_categorical_crossentropy',
        'district_output': 'sparse_categorical_crossentropy',
        'street_output': 'sparse_categorical_crossentropy'
    },
    metrics={
        'city_output': 'accuracy',
        'district_output': 'accuracy',
        'street_output': 'accuracy'
    }
)

model.summary()


Krok 4: Budowa i Kompilacja Modelu (Zoptymalizowana Architektura)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ text_input (InputLayer)       │ (None, 5000)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ num_input (InputLayer)        │ (None, 2)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 128)               │         640,128 │ text_input[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_1 (Dense)               │ (None, 64)                │             192 │ num_input[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout (Dropout)             │ (None, 128)               │               0 │ dense[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_2 (Dense)               │ (None, 32)                │           2,080 │ dense_1[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 160)               │               0 │ dropout[0][0],             │
│                               │                           │                 │ dense_2[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_3 (Dense)               │ (None, 256)               │          41,216 │ concatenate[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_1 (Dropout)           │ (None, 256)               │               0 │ dense_3[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ city_output (Dense)           │ (None, 6933)              │       1,781,781 │ dropout_1[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ district_output (Dense)       │ (None, 1940)              │         498,580 │ dropout_1[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ street_output (Dense)         │ (None, 28177)             │       7,241,489 │ dropout_1[0][0]            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 10,205,466 (38.93 MB)

 Trainable params: 10,205,466 (38.93 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
print("\nKrok 5: Trening Modelu")

# Przygotowanie danych do treningu
X_train_list = [X_text_train, X_num_train]
y_train_dict = {'city_output': y_city_train, 'district_output': y_district_train, 'street_output': y_street_train}

X_val_list = [X_text_val, X_num_val]
y_val_dict = {'city_output': y_city_val, 'district_output': y_district_val, 'street_output': y_street_val}

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Trening
history = model.fit(
    X_train_list,
    y_train_dict,
    validation_data=(X_val_list, y_val_dict),
    epochs=25,
    batch_size=256,
    callbacks=[early_stopping]
)

# Zapis modelu
model.save(os.path.join(ARTIFACTS_DIR, 'location_prediction_model.h5'))
print("Model został wytrenowany i zapisany.")


Krok 5: Trening Modelu
Epoch 1/25
2216/2216 ━━━━━━━━━━━━━━━━━━━━ 228s 102ms/step - city_output_accuracy: 0.2689 - city_output_loss: 4.3503 - district_output_accuracy: 0.4954 - district_output_loss: 3.6707 - loss: 12.1831 - street_output_accuracy: 0.6624 - street_output_loss: 4.1621 - val_city_output_accuracy: 0.5995 - val_city_output_loss: 2.4684 - val_district_output_accuracy: 0.5279 - val_district_output_loss: 2.3536 - val_loss: 7.9616 - val_street_output_accuracy: 0.6713 - val_street_output_loss: 3.1396
Epoch 2/25
2216/2216 ━━━━━━━━━━━━━━━━━━━━ 222s 100ms/step - city_output_accuracy: 0.5868 - city_output_loss: 2.4949 - district_output_accuracy: 0.5206 - district_output_loss: 2.3802 - loss: 8.0498 - street_output_accuracy: 0.6697 - street_output_loss: 3.1748 - val_city_output_accuracy: 0.6508 - val_city_output_loss: 2.0860 - val_district_output_accuracy: 0.5632 - val_district_output_loss: 1.9731 - val_loss: 6.8947 - val_street_output_accuracy: 0.6721 - val_street_output_loss: 2.8357

Model został wytrenowany i zapisany.


In [11]:
import gc
from tqdm.notebook import tqdm

print("\nKrok 7: Predykcja i Zastosowanie Logiki Hierarchicznej (Przetwarzanie w locie)")

n_samples = df.shape[0]
chunk_size = 50000  # Bezpieczny, mały rozmiar kawałka

predicted_city_ids = []
predicted_district_ids = []
predicted_street_ids = []

print(f"Rozpoczynam predykcję na {n_samples} próbkach w kawałkach po {chunk_size}...")

# Zewnętrzna pętla po kawałkach danych wejściowych
for i in tqdm(range(0, n_samples, chunk_size)):
    chunk_text = X_text[i:i + chunk_size]
    chunk_num = X_num[i:i + chunk_size]
    chunk_list = [chunk_text, chunk_num]
    
    # POPRAWKA: Używamy nazwy `model`, tak jak został zdefiniowany i wytrenowany w tym notatniku
    pred_city_chunk, pred_district_chunk, pred_street_chunk = model.predict(chunk_list, batch_size=512, verbose=0)
    
    # Wewnętrzna pętla do przetwarzania wyników z tego kawałka NATYCHMIAST
    for j in range(pred_city_chunk.shape[0]):
        city_pred_index = np.argmax(pred_city_chunk[j])
        city_id = inv_city_id_map.get(city_pred_index, 0)
        
        valid_districts_for_city = city_to_districts.get(city_id, [])
        district_pred_index = np.argmax(pred_district_chunk[j]) 
        if valid_districts_for_city:
            district_mask = np.zeros_like(pred_district_chunk[j])
            valid_indices = [district_id_map.get(d_id) for d_id in valid_districts_for_city if district_id_map.get(d_id) is not None]
            if valid_indices:
                district_mask[valid_indices] = 1
                if np.sum(district_mask) > 0:
                    district_pred_index = np.argmax(pred_district_chunk[j] * district_mask)
        
        district_id = inv_district_id_map.get(district_pred_index, 0)
        
        valid_streets_for_district = district_to_streets.get(district_id, [])
        street_pred_index = np.argmax(pred_street_chunk[j]) 
        if valid_streets_for_district:
            street_mask = np.zeros_like(pred_street_chunk[j])
            valid_indices = [street_id_map.get(s_id) for s_id in valid_streets_for_district if street_id_map.get(s_id) is not None]
            if valid_indices:
                street_mask[valid_indices] = 1
                if np.sum(street_mask) > 0:
                    street_pred_index = np.argmax(pred_street_chunk[j] * street_mask)

        street_id = inv_street_id_map.get(street_pred_index, 0)
        
        predicted_city_ids.append(city_id)
        predicted_district_ids.append(district_id)
        predicted_street_ids.append(street_id)
        
    # Ręczne zwolnienie pamięci po każdej pętli
    del pred_city_chunk, pred_district_chunk, pred_street_chunk
    gc.collect()

# Na koniec, dodajemy skompletowane listy do ramki danych
df['predicted_city_id'] = predicted_city_ids
df['predicted_district_id'] = predicted_district_ids
df['predicted_street_id'] = predicted_street_ids

print("Zakończono predykcję z logiką hierarchiczną.")


Krok 7: Predykcja i Zastosowanie Logiki Hierarchicznej (Przetwarzanie w locie)
Rozpoczynam predykcję na 709104 próbkach w kawałkach po 50000...


  0%|          | 0/15 [00:00<?, ?it/s]

Zakończono predykcję z logiką hierarchiczną.


In [12]:
print("Odtwarzam ramkę danych 'df_original'...")

# Ponownie wczytujemy dane w ten sam sposób, co w komórce 3,
# aby mieć pewność, że df_original ma identyczną strukturę jak df przed filtrowaniem.

# Szybki odczyt pierwszego wiersza, żeby znaleźć ostatnią kolumnę
try:
    first_row = pd.read_csv('saleflats_2024_dateAdded_polska.csv', sep=',', header=None, nrows=1, on_bad_lines='skip')
    last_col_index = first_row.shape[1] - 1
except:
    last_col_index = 64 # Wartość awaryjna

COLS_TO_EXTRACT = {
    3: 'Title', 4: 'Description', 5: 'Area', 6: 'Price',
    9: 'NumberOfRooms', 14: 'Floor', 15: 'Floors', 16: 'BuiltYear',
    last_col_index: 'locationPath'
}

all_offers_df_orig = []
FILES = ['saleflats_2024_dateAdded_polska.csv', 'saleflats_2024_newestDate_polska.csv']
for filepath in FILES:
    try:
        df_chunk = pd.read_csv(
            filepath, sep=',', header=None, on_bad_lines='skip',
            usecols=list(COLS_TO_EXTRACT.keys()), low_memory=False, dtype=str, encoding='utf-8'
        )
        df_chunk.rename(columns=COLS_TO_EXTRACT, inplace=True)
        all_offers_df_orig.append(df_chunk)
    except Exception:
        pass # Ignorujemy błędy, bo już raz wczytaliśmy

df_original = pd.concat(all_offers_df_orig, ignore_index=True)
df_original.drop_duplicates(subset=['Title', 'Description', 'locationPath'], keep='first', inplace=True)

print("'df_original' został pomyślnie odtworzony.")

Odtwarzam ramkę danych 'df_original'...
'df_original' został pomyślnie odtworzony.


In [13]:
print("\nKrok 7: Generowanie i Zapis Wyników")

def create_loc_string(row):
    city = id_to_name.get(row['predicted_city_id'], "?")
    district = id_to_name.get(row['predicted_district_id'], "?")
    street = id_to_name.get(row['predicted_street_id'], "?")
    
    # Zwracamy "?" dla nieokreślonych (ID=0)
    if row['predicted_district_id'] == 0: district = '?'
    if row['predicted_street_id'] == 0: street = '?'
        
    return f"{city} > {district} > {street}"

df['Predict_Loc'] = df.apply(create_loc_string, axis=1)

# Przygotowujemy do scalenia z oryginalnym DF
df_to_merge = df[['Title', 'Description', 'Predict_Loc']].copy()
# Tworzymy unikalny klucz do merge'owania, żeby uniknąć problemów
df_to_merge['merge_key'] = df_to_merge['Title'].fillna('') + df_to_merge['Description'].fillna('')
df_original['merge_key'] = df_original['Title'].fillna('') + df_original['Description'].fillna('')

# Scalenie
df_final = pd.merge(df_original, df_to_merge[['merge_key', 'Predict_Loc']], on='merge_key', how='left')
df_final.drop(columns=['merge_key'], inplace=True, errors='ignore')
df_final.drop_duplicates(subset=['Title', 'Description'], inplace=True)
df_final['Predict_Loc'].fillna('Brak predykcji', inplace=True)

# Zapis do pliku CSV
OUTPUT_FILE = 'Location_Polska.csv'
df_final.to_csv(OUTPUT_FILE, index=False, sep=';', encoding='utf-8-sig')
print(f"Wyniki zostały zapisane do pliku: {OUTPUT_FILE}")


Krok 7: Generowanie i Zapis Wyników
Wyniki zostały zapisane do pliku: Location_Polska.csv


In [14]:
print("\nPrzykładowe 20 wierszy z wynikami:")

predicted_df = df_final[df_final['Predict_Loc'] != 'Brak predykcji']
if len(predicted_df) > 20:
    sample = predicted_df.sample(20)
else:
    sample = predicted_df.head(20)

display_cols = ['Title', 'locationPath', 'Predict_Loc']
sample_display = sample[display_cols]

def highlight_col(s):
    return ['background-color: #ffff99' if c == 'Predict_Loc' else '' for c in s.index]

styled_sample = sample_display.style.apply(highlight_col, axis=1)

display(styled_sample)


Przykładowe 20 wierszy z wynikami:


,Title,locationPath,Predict_Loc
94197,Kraków Prądnik Biały Zawodzie 55m 2pok Balkon,"14,0,0,337,3257,85909,0",Kraków > Prądnik biały > ?
299380,Wyjątkowy 4 pokojowy apartament z tarasem,"9,326,0,10547,0,0,0",Warszawa > Wilanów > ?
694391,"Mieszkanie Olsztyn Nagórki, ul. Franciszka Barcza Olsztyn","2,202,0,27489,0,0,0",Olsztyn > ? > ?
245088,"Mieszkanie Elbląg, ul. Świerkowa","6,0,0,349,0,0,314877",Elbląg > ? > ?
755727,Sprzedam mieszkanie,"8,325,0,7364,0,0,0",Żyrardów > ? > ?
354963,"Mieszkanie, Dywity, Dywity (gm.), 46 m²","6,194,0,54323,0,0,0",Olsztyn > ? > ?
297359,Mieszkanie 39 m2 1 piętro z 4,"9,0,0,332,0,0,0",Legnica > ? > ?
447571,Mieszkanie - Warszawa Wola,"9,0,0,368,3264,98730,269147",Warszawa > Wola > ?
206764,"Mieszkanie, Warszawa, Śródmieście, 57 m²","9,0,0,368,0,0,0",Warszawa > Śródmieście > ?
217710,"3 pokoje, Blisko centrum, niski czynsz, Piwnica","11,0,0,373,0,84042,488646",Gdańsk > ? > ?
